# Rainfall Prediction Project

## Data Loading, combining and EDA

*Group 12*

------------

In [8]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## Download the data

In [9]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data/raw/figsharerainfall/"

In [10]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]

In [11]:
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [12]:
with zipfile.ZipFile(os.path.join(output_directory, files_to_dl[0]), 'r') as f:
    f.extractall(output_directory)

## Combine the data

In [13]:
%%time
exclude = "observed_daily_rainfall_SYD.csv"
files = glob.glob('../data/raw/figsharerainfall/*.csv')
df = pd.concat(
    (pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'[A-Z][^_]+', file)[0])
                for file in files if file is not exclude)
)
df.to_csv("../data/processed/combined_data.csv")

CPU times: user 5min 49s, sys: 7.46 s, total: 5min 57s
Wall time: 6min


In [16]:
df = pd.read_csv("../data/processed/combined_data.csv")
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [17]:
df.tail()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
62513858,2014-12-27 12:00:00,-30.157068,-29.21466,153.125,154.375,6.689683,SAM0-UNICON
62513859,2014-12-28 12:00:00,-30.157068,-29.21466,153.125,154.375,7.862555,SAM0-UNICON
62513860,2014-12-29 12:00:00,-30.157068,-29.21466,153.125,154.375,10.005026,SAM0-UNICON
62513861,2014-12-30 12:00:00,-30.157068,-29.21466,153.125,154.375,8.541592,SAM0-UNICON
62513862,2014-12-31 12:00:00,-30.157068,-29.21466,153.125,154.375,68.117489,SAM0-UNICON


## Combine data csv on different machines

- Compare observations:

| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      | macOS            | 16GB| M1        | No     | 6min 39s   |
| Lynn Wu       | macOS            | 8GB | M1        | Yes    |  6min 7s   |
| Jasmine Ortega|                  |     |           |        |            |
| Maeve Shi   | MacOS Big Sur    | 8GB | 2.3 GHz Dual-Core Intel Core i5 | Yes |  7min 30s   |

--------------

##  Load csv and perform EDA on different machines

- Approaches to reduce memory usage while performing the EDA:

    - 
    - 

- Compare observations:

| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      |                  |     |           |        |            |
| Lynn Wu       |                  |     |           |        |            |
| Jasmine Ortega|                  |     |           |        |            |
| Yike Shi      |                  |     |           |        |            |